# 用于欺骗相关模型产生预测

In [1]:
import os
from tqdm import tqdm
from pycocotools.coco import COCO
from mmengine import load, dump

#### keypoint

In [2]:
# body2d_categories = load('/data0/DataSet/COCO2014/annotations/person_keypoints_val2014.json')['categories']
# dump(body2d_categories, '../../configs/pose_categories.json')
body2d_categories = load('../../configs/pose_categories.json')

def gen_fake_body2d_keypoint(coco_ins: COCO):
    annotations = []
    for img_id in tqdm(coco_ins.getImgIds()):
        annotations.append({
            'keypoints': [],
            'num_keypoints': 17,
            'id': img_id,
            'image_id': img_id,
            'category_id': 1,
            'area': 100,
            'bbox': [10, 10, 20, 20],
            'iscrowd': 0,
        })
    body2d = {
        "info": coco_ins.dataset['info'],
        "licenses": coco_ins.dataset['licenses'],
        "images": coco_ins.dataset['images'],
        "annotations": annotations,
        "categories": body2d_categories
    }
    return body2d

In [3]:
# wholebody_categories = load('pose_example/wholebody_example_gt.json')['categories']
# dump(wholebody_categories, '../../configs/wholebody_categories.json')
wholebody_categories = load('../../configs/wholebody_categories.json')

def gen_fake_wholebody_keypoint(coco_ins: COCO):
    annotations = []
    for img_id in tqdm(coco_ins.getImgIds()):
        annotations.append({
            'keypoints': [],
            'foot_kpts': [],
            'face_kpts': [],
            'lefthand_kpts': [],
            'righthand_kpts': [],
            'num_keypoints': 17,
            'id': img_id,
            'image_id': img_id,
            'category_id': 1,
            'area': 100,
            'bbox': [10, 10, 20, 20],
            'face_box': [10, 10, 20, 20],
            'lefthand_box': [10, 10, 20, 20],
            'righthand_box': [10, 10, 20, 20],
            'iscrowd': 0,
            'face_valid': True,
            'lefthand_valid': True,
            'righthand_valid': True,
            'foot_valid': True,
        })
    wholebody = {
        "info": coco_ins.dataset['info'],
        "licenses": coco_ins.dataset['licenses'],
        "images": coco_ins.dataset['images'],
        "annotations": annotations,
        "categories": wholebody_categories
    }
    return wholebody

#### panoptic

In [4]:
# panoptic_categories = load('/data0/DataSet/COCO2017/annotations/panoptic_val2017.json')['categories']
# dump(panoptic_categories, '../../configs/panoptic_categories.json')
panoptic_categories = load('../../configs/panoptic_categories.json')

def gen_fake_panoptic(coco_ins: COCO):
    annotations = []
    for img_id in tqdm(coco_ins.getImgIds()):
        annotations.append({
            'segments_info':[{
                'id': img_id,
                'category_id': 1,
                'iscrowd': 0,
                'bbox': [10, 10, 20, 20],
                'area': 100
            }],
            # 需要运行gray2rgb.py，将gray图片转为rgb图片
            'file_name': coco_ins.loadImgs(img_id)[0]['file_name'],  
            'image_id': img_id
        })
    new_anns = {
        "info": coco_ins.dataset['info'],
        "licenses": coco_ins.dataset['licenses'],
        "images": coco_ins.dataset['images'],
        "annotations": annotations,
        "categories": panoptic_categories
    }
    return new_anns

## func map

In [5]:
func_map = {
    'person_keypoints': gen_fake_body2d_keypoint,
    'person_wholebody': gen_fake_wholebody_keypoint,
    'panoptics': gen_fake_panoptic
}

# generate

In [6]:
fake_root = '../../annotations/fake/'
if not os.path.exists(fake_root):
    os.makedirs(fake_root)

In [8]:
coco_ins = COCO('../../annotations/instances_train.json')

for type_name, func in func_map.items():
    coco_fake = func(coco_ins)
    dump(coco_fake, os.path.join(fake_root, f'fake_{type_name}_train.json'))

  0%|          | 0/5400 [00:00<?, ?it/s]

100%|██████████| 5400/5400 [00:00<00:00, 534621.54it/s]


In [9]:
coco_ins = COCO('../../annotations/instances_test.json')
for type_name, func in func_map.items():
    coco_fake = func(coco_ins)
    dump(coco_fake, os.path.join(fake_root, f'fake_{type_name}_test.json'))

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


100%|██████████| 4946/4946 [00:00<00:00, 573574.09it/s]
